In [11]:
import numpy as np
import pandas as pd,glob, os
import time
import re
from functools import reduce
from tqdm.auto import tqdm

In [16]:
TEAM_RS_PATH = "dataset/team/team_stats_*_rs.csv"

In [19]:
def read_team_rs():
    dfs = []
    for f in glob.glob(TEAM_RS_PATH):
        key = os.path.basename(f).split("_")[2]              # e.g. 'advanced'
        df = pd.read_csv(f)
        # simplify column names & prefix by table type to avoid collisions
        df.columns = [c.lower().strip()                       # lowercase
                      if c.lower() in {"team_id", "team_name", "season"}
                      else f"{key}__{c.lower().strip()}"
                      for c in df.columns]
        dfs.append(df)
    # inner-join on the common keys
    team_rs = reduce(lambda left, right:
                     pd.merge(left, right,
                              on=["team_id", "team_name", "season"]),
                     dfs)
    return team_rs

In [20]:
team_rs = read_team_rs()
print(team_rs.shape)

(802, 295)


In [40]:
df_2023 = team_rs[team_rs["season"] == '2020-21'].copy()

In [41]:
df_2023.head()

,team_id,team_name,misc__gp,misc__w,misc__l,misc__w_pct,misc__min,misc__pts_off_tov,misc__pts_2nd_chance,misc__pts_fb,...,traditional__reb_rank,traditional__ast_rank,traditional__tov_rank,traditional__stl_rank,traditional__blk_rank,traditional__blka_rank,traditional__pf_rank,traditional__pfd_rank,traditional__pts_rank,traditional__plus_minus_rank
712,1610612737,Atlanta Hawks,72,41,31,0.569,48.3,14.7,14.3,10.0,...,5,19,9,24,17,17,18,9,13,9
713,1610612738,Boston Celtics,72,36,36,0.500,48.3,17.8,13.3,11.8,...,15,25,16,13,6,11,24,13,16,13
714,1610612751,Brooklyn Nets,72,48,24,0.667,48.3,15.8,12.3,14.3,...,14,7,13,27,7,9,13,18,2,7
715,1610612766,Charlotte Hornets,72,33,39,0.458,48.2,17.7,12.7,13.6,...,19,5,24,10,16,14,5,21,23,23
716,1610612741,Chicago Bulls,72,31,41,0.431,48.3,14.5,13.2,11.5,...,11,8,27,28,27,18,10,30,21,20


In [42]:
team_rs["season_start"] = (
    team_rs["season"]
      .str.split("-", n=1)
      .str[0]
      .astype(int)          # 2021-22 → 2021
)

team_rs = team_rs.sort_values("season_start").reset_index(drop=True)

In [43]:
team_rs.head()

,team_id,team_name,misc__gp,misc__w,misc__l,misc__w_pct,misc__min,misc__pts_off_tov,misc__pts_2nd_chance,misc__pts_fb,...,traditional__ast_rank,traditional__tov_rank,traditional__stl_rank,traditional__blk_rank,traditional__blka_rank,traditional__pf_rank,traditional__pfd_rank,traditional__pts_rank,traditional__plus_minus_rank,season_start
0,1610612737,Atlanta Hawks,82,56,26,0.683,48.3,17.8,13.2,8.9,...,28,7,9,11,4,1,27,23,5,1996
1,1610612764,Washington Bullets,82,44,38,0.537,48.4,18.7,14.8,14.5,...,7,14,8,13,5,15,8,10,13,1996
2,1610612763,Vancouver Grizzlies,82,14,68,0.171,48.2,15.4,14.0,11.2,...,10,16,15,6,26,9,27,28,29,1996
3,1610612762,Utah Jazz,82,64,18,0.780,48.4,19.2,12.7,14.0,...,1,12,3,12,10,28,6,2,2,1996
4,1610612761,Toronto Raptors,82,30,52,0.366,48.4,17.8,15.0,17.5,...,19,23,6,3,22,20,26,18,20,1996


In [48]:
po_seeds = (
    pd.read_csv("dataset/team/team_stats_traditional_po.csv")
      .loc[:, ["SEASON", "TEAM_ID"]]
      .drop_duplicates()
)


In [54]:
po_seeds.head()

,SEASON,TEAM_ID
0,1996-97,1610612737
1,1996-97,1610612766
2,1996-97,1610612741
3,1996-97,1610612765
4,1996-97,1610612745


In [55]:
team_rs.head()

,team_id,team_name,misc__gp,misc__w,misc__l,misc__w_pct,misc__min,misc__pts_off_tov,misc__pts_2nd_chance,misc__pts_fb,...,traditional__ast_rank,traditional__tov_rank,traditional__stl_rank,traditional__blk_rank,traditional__blka_rank,traditional__pf_rank,traditional__pfd_rank,traditional__pts_rank,traditional__plus_minus_rank,season_start
0,1610612737,Atlanta Hawks,82,56,26,0.683,48.3,17.8,13.2,8.9,...,28,7,9,11,4,1,27,23,5,1996
1,1610612764,Washington Bullets,82,44,38,0.537,48.4,18.7,14.8,14.5,...,7,14,8,13,5,15,8,10,13,1996
2,1610612763,Vancouver Grizzlies,82,14,68,0.171,48.2,15.4,14.0,11.2,...,10,16,15,6,26,9,27,28,29,1996
3,1610612762,Utah Jazz,82,64,18,0.780,48.4,19.2,12.7,14.0,...,1,12,3,12,10,28,6,2,2,1996
4,1610612761,Toronto Raptors,82,30,52,0.366,48.4,17.8,15.0,17.5,...,19,23,6,3,22,20,26,18,20,1996


In [60]:
team_rs.rename(columns={"season": "SEASON"}, inplace=True)
team_rs.rename(columns={"team_id": "TEAM_ID"}, inplace=True)

In [61]:
team_rs.head()

,TEAM_ID,team_name,misc__gp,misc__w,misc__l,misc__w_pct,misc__min,misc__pts_off_tov,misc__pts_2nd_chance,misc__pts_fb,...,traditional__ast_rank,traditional__tov_rank,traditional__stl_rank,traditional__blk_rank,traditional__blka_rank,traditional__pf_rank,traditional__pfd_rank,traditional__pts_rank,traditional__plus_minus_rank,season_start
0,1610612737,Atlanta Hawks,82,56,26,0.683,48.3,17.8,13.2,8.9,...,28,7,9,11,4,1,27,23,5,1996
1,1610612764,Washington Bullets,82,44,38,0.537,48.4,18.7,14.8,14.5,...,7,14,8,13,5,15,8,10,13,1996
2,1610612763,Vancouver Grizzlies,82,14,68,0.171,48.2,15.4,14.0,11.2,...,10,16,15,6,26,9,27,28,29,1996
3,1610612762,Utah Jazz,82,64,18,0.780,48.4,19.2,12.7,14.0,...,1,12,3,12,10,28,6,2,2,1996
4,1610612761,Toronto Raptors,82,30,52,0.366,48.4,17.8,15.0,17.5,...,19,23,6,3,22,20,26,18,20,1996


In [63]:
team_rs = team_rs.merge(po_seeds, on=["SEASON", "TEAM_ID"], how="inner")
print(team_rs["SEASON"].value_counts())  # should show 16 rows / season


SEASON
1996-97    16
2010-11    16
2021-22    16
2020-21    16
2019-20    16
2018-19    16
2017-18    16
2016-17    16
2015-16    16
2014-15    16
2013-14    16
2012-13    16
2011-12    16
2009-10    16
1997-98    16
2008-09    16
2007-08    16
2006-07    16
2005-06    16
2004-05    16
2003-04    16
2002-03    16
2001-02    16
2000-01    16
1999-00    16
1998-99    16
2022-23    16
Name: count, dtype: int64


In [64]:
team_rs.columns = [
    c.lower()                     # or any transformation you need
    for c in team_rs.columns
]

In [66]:
pd.set_option("display.max_columns", None)

In [68]:
import pandas as pd 
pd.set_option("display.max_seq_items", None)

In [69]:
team_rs.columns 

Index(['team_id', 'team_name', 'misc__gp', 'misc__w', 'misc__l', 'misc__w_pct',
       'misc__min', 'misc__pts_off_tov', 'misc__pts_2nd_chance',
       'misc__pts_fb', 'misc__pts_paint', 'misc__opp_pts_off_tov',
       'misc__opp_pts_2nd_chance', 'misc__opp_pts_fb', 'misc__opp_pts_paint',
       'misc__gp_rank', 'misc__w_rank', 'misc__l_rank', 'misc__w_pct_rank',
       'misc__min_rank', 'misc__pts_off_tov_rank', 'misc__pts_2nd_chance_rank',
       'misc__pts_fb_rank', 'misc__pts_paint_rank',
       'misc__opp_pts_off_tov_rank', 'misc__opp_pts_2nd_chance_rank',
       'misc__opp_pts_fb_rank', 'misc__opp_pts_paint_rank', 'season',
       'advanced__gp', 'advanced__w', 'advanced__l', 'advanced__w_pct',
       'advanced__min', 'advanced__e_off_rating', 'advanced__off_rating',
       'advanced__e_def_rating', 'advanced__def_rating',
       'advanced__e_net_rating', 'advanced__net_rating', 'advanced__ast_pct',
       'advanced__ast_to', 'advanced__ast_ratio', 'advanced__oreb_pct',
       'a

In [72]:
team_rs.head()

,team_id,team_name,misc__gp,misc__w,misc__l,misc__w_pct,misc__min,misc__pts_off_tov,misc__pts_2nd_chance,misc__pts_fb,misc__pts_paint,misc__opp_pts_off_tov,misc__opp_pts_2nd_chance,misc__opp_pts_fb,misc__opp_pts_paint,misc__gp_rank,misc__w_rank,misc__l_rank,misc__w_pct_rank,misc__min_rank,misc__pts_off_tov_rank,misc__pts_2nd_chance_rank,misc__pts_fb_rank,misc__pts_paint_rank,misc__opp_pts_off_tov_rank,misc__opp_pts_2nd_chance_rank,misc__opp_pts_fb_rank,misc__opp_pts_paint_rank,season,advanced__gp,advanced__w,advanced__l,advanced__w_pct,advanced__min,advanced__e_off_rating,advanced__off_rating,advanced__e_def_rating,advanced__def_rating,advanced__e_net_rating,advanced__net_rating,advanced__ast_pct,advanced__ast_to,advanced__ast_ratio,advanced__oreb_pct,advanced__dreb_pct,advanced__reb_pct,advanced__tm_tov_pct,advanced__efg_pct,advanced__ts_pct,advanced__e_pace,advanced__pace,advanced__pace_per40,advanced__poss,advanced__pie,advanced__gp_rank,advanced__w_rank,advanced__l_rank,advanced__w_pct_rank,advanced__min_rank,advanced__off_rating_rank,advanced__def_rating_rank,advanced__net_rating_rank,advanced__ast_pct_rank,advanced__ast_to_rank,advanced__ast_ratio_rank,advanced__oreb_pct_rank,advanced__dreb_pct_rank,advanced__reb_pct_rank,advanced__tm_tov_pct_rank,advanced__efg_pct_rank,advanced__ts_pct_rank,advanced__pace_rank,advanced__pie_rank,four__gp,four__w,four__l,four__w_pct,four__min,four__efg_pct,four__fta_rate,four__tm_tov_pct,four__oreb_pct,four__opp_efg_pct,four__opp_fta_rate,four__opp_tov_pct,four__opp_oreb_pct,four__gp_rank,four__w_rank,four__l_rank,four__w_pct_rank,four__min_rank,four__efg_pct_rank,four__fta_rate_rank,four__tm_tov_pct_rank,four__oreb_pct_rank,four__opp_efg_pct_rank,four__opp_fta_rate_rank,four__opp_tov_pct_rank,four__opp_oreb_pct_rank,scoring__gp,scoring__w,scoring__l,scoring__w_pct,scoring__min,scoring__pct_fga_2pt,scoring__pct_fga_3pt,scoring__pct_pts_2pt,scoring__pct_pts_2pt_mr,scoring__pct_pts_3pt,scoring__pct_pts_fb,scoring__pct_pts_ft,scoring__pct_pts_off_tov,scoring__pct_pts_paint,scoring__pct_ast_2pm,scoring__pct_uast_2pm,scoring__pct_ast_3pm,scoring__pct_uast_3pm,scoring__pct_ast_fgm,scoring__pct_uast_fgm,scoring__gp_rank,scoring__w_rank,scoring__l_rank,scoring__w_pct_rank,scoring__min_rank,scoring__pct_fga_2pt_rank,scoring__pct_fga_3pt_rank,scoring__pct_pts_2pt_rank,scoring__pct_pts_2pt_mr_rank,scoring__pct_pts_3pt_rank,scoring__pct_pts_fb_rank,scoring__pct_pts_ft_rank,scoring__pct_pts_off_tov_rank,scoring__pct_pts_paint_rank,scoring__pct_ast_2pm_rank,scoring__pct_uast_2pm_rank,scoring__pct_ast_3pm_rank,scoring__pct_uast_3pm_rank,scoring__pct_ast_fgm_rank,scoring__pct_uast_fgm_rank,defense__gp,defense__w,defense__l,defense__w_pct,defense__min,defense__opp_fgm,defense__opp_fga,defense__opp_fg_pct,defense__opp_fg3m,defense__opp_fg3a,defense__opp_fg3_pct,defense__opp_ftm,defense__opp_fta,defense__opp_ft_pct,defense__opp_oreb,defense__opp_dreb,defense__opp_reb,defense__opp_ast,defense__opp_tov,defense__opp_stl,defense__opp_blk,defense__opp_blka,defense__opp_pf,defense__opp_pfd,defense__opp_pts,defense__plus_minus,defense__gp_rank,defense__w_rank,defense__l_rank,defense__w_pct_rank,defense__min_rank,defense__opp_fgm_rank,defense__opp_fga_rank,defense__opp_fg_pct_rank,defense__opp_fg3m_rank,defense__opp_fg3a_rank,defense__opp_fg3_pct_rank,defense__opp_ftm_rank,defense__opp_fta_rank,defense__opp_ft_pct_rank,defense__opp_oreb_rank,defense__opp_dreb_rank,defense__opp_reb_rank,defense__opp_ast_rank,defense__opp_tov_rank,defense__opp_stl_rank,defense__opp_blk_rank,defense__opp_blka_rank,defense__opp_pf_rank,defense__opp_pfd_rank,defense__opp_pts_rank,defense__plus_minus_rank,opponent__gp,opponent__w,opponent__l,opponent__w_pct,opponent__min,opponent__opp_fgm,opponent__opp_fga,opponent__opp_fg_pct,opponent__opp_fg3m,opponent__opp_fg3a,opponent__opp_fg3_pct,opponent__opp_ftm,opponent__opp_fta,opponent__opp_ft_pct,opponent__opp_oreb,opponent__opp_dreb,opponent__opp_reb,opponent__opp_ast,opponent__opp_tov,oppo